In [2]:
import pandas as pd
import numpy as np
import os
import evaluate

from datasets import Audio, Dataset, ClassLabel
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments, Trainer, pipeline


ModuleNotFoundError: No module named 'evaluate'

In [3]:
audio_paths = os.listdir("../data/audios/")
audio_paths = ["..data/audios/"+audio for audio in audio_paths]
df_finetuning = pd.DataFrame({"path": audio_paths})
df_finetuning["label"] = df_finetuning["path"].apply(lambda x: 1 if "_AI." in x else 0)

In [ ]:
ds = Dataset.from_pandas(df_finetuning)
ds = ds.cast_column("path", Audio(sampling_rate=16000))
ds = ds.cast_column("label", ClassLabel(names=["human", "AI"]))
ds = ds.train_test_split(seed=42, shuffle=True, test_size=0.2)

In [ ]:
model_id = "facebook/wav2vec2-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["path"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * 10),
        truncation=True,
        padding=True,
        return_tensors="pt",
    )
    return inputs

In [ ]:
ds_encoded = ds.map(
    preprocess_function,
    remove_columns=["path"],
    batched=True,
    batch_size=100,
    num_proc=1,
)

In [ ]:
id2label_fn = ds["train"].features["label"].int2str
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(ds_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

In [ ]:
model_name = model_id.split("/")[-1]
batch_size = 32
gradient_accumulation_steps = 4
num_train_epochs = 5

training_args = TrainingArguments(
    output_dir = f"{model_name}-scam-es-common_voice-elevenlabs",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to="none"
)

In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=ds_encoded["train"],
    eval_dataset=ds_encoded["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()